In [13]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [14]:
#This dictionary is used to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [15]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame is:
    DataFrame( columns=["State", "RegionName"]  )
    
    The following cleaning is done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.'''
    def exstate(data):
        if(data.loc[0].split('[')[0] in states.values()):
            data['State']=data.loc[0].split('[')[0]
            data.loc[0]=data.loc[0].split('[')[0]
        return data
    utowns=pd.read_csv('university_towns.txt',sep="/n",header=None)
    utowns=utowns.apply(exstate,axis=1).fillna(method='ffill')
    utowns['RegionName']=utowns[utowns[0]!=utowns['State']].apply(lambda x: x[0].split('(')[0].strip(),axis=1)
    utowns=utowns.dropna().drop(0,axis=1).reset_index().drop(['index'],axis=1)
    return utowns

In [16]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp=pd.read_excel('gdplev.xls',skiprows=5,header=0,usecols=[4,6]).dropna()
    gdp.set_index('Unnamed: 4',inplace=True)
    gdp=gdp.loc['2000q1':]
    gdp=gdp.rename(columns={'GDP in billions of chained 2009 dollars.1':'GDP'})
    gdp.index.rename('Year',inplace=True)
    for i in range(2,(len(gdp.index)-2)):
        j,flagd=0,0
        while(j<=i-1):
            if(gdp.iloc[i-j-1,0]>gdp.iloc[i-j,0]):
                flagd+=1
            else:
                break
            j+=1
        if(flagd>=2):
            k,flagu=0,0
            while(k<2):
                if(gdp.iloc[i+k,0]<gdp.iloc[i+k+1,0]):
                    flagu+=1
                k+=1
            if(flagu==2):
                start=i-flagd+1
                bottom=i
                end=i+flagu
                break
        else:
            continue
    return gdp.index.values[start]

In [17]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp=pd.read_excel('gdplev.xls',skiprows=5,header=0,usecols=[4,6]).dropna()
    gdp.set_index('Unnamed: 4',inplace=True)
    gdp=gdp.loc['2000q1':]
    gdp=gdp.rename(columns={'GDP in billions of chained 2009 dollars.1':'GDP'})
    gdp.index.rename('Year',inplace=True)
    for i in range(2,(len(gdp.index)-2)):
        j,flagd=0,0
        while(j<=i-1):
            if(gdp.iloc[i-j-1,0]>gdp.iloc[i-j,0]):
                flagd+=1
            else:
                break
            j+=1
        if(flagd>=2):
            k,flagu=0,0
            while(k<2):
                if(gdp.iloc[i+k,0]<gdp.iloc[i+k+1,0]):
                    flagu+=1
                k+=1
            if(flagu==2):
                start=i-flagd+1
                bottom=i
                end=i+flagu
                break
        else:
            continue
    return gdp.index.values[end]

In [18]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp=pd.read_excel('gdplev.xls',skiprows=5,header=0,usecols=[4,6]).dropna()
    gdp.set_index('Unnamed: 4',inplace=True)
    gdp=gdp.loc['2000q1':]
    gdp=gdp.rename(columns={'GDP in billions of chained 2009 dollars.1':'GDP'})
    gdp.index.rename('Year',inplace=True)
    for i in range(2,(len(gdp.index)-2)):
        j,flagd=0,0
        while(j<=i-1):
            if(gdp.iloc[i-j-1,0]>gdp.iloc[i-j,0]):
                flagd+=1
            else:
                break
            j+=1
        if(flagd>=2):
            k,flagu=0,0
            while(k<2):
                if(gdp.iloc[i+k,0]<gdp.iloc[i+k+1,0]):
                    flagu+=1
                k+=1
            if(flagu==2):
                start=i-flagd+1
                bottom=i
                end=i+flagu
                break
        else:
            continue
    return gdp.index.values[bottom]

In [19]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe has a multi-index
    in the shape of ["State","RegionName"].
    '''
    homes=pd.read_csv('City_Zhvi_AllHomes.csv')
    homes=pd.merge(homes[['State','RegionName']],homes.loc[:,'2000-01':],left_index=True,right_index=True)
    homes['State']=homes.apply(lambda x:states[x['State']],axis=1)
    homes=homes.set_index(['State','RegionName'])
    homes.columns=pd.to_datetime(homes.columns)
    homes=homes.resample('Q',axis=1).mean()
    homes.columns=[str(homes.columns[i]).split('-')[0]+'q'+str(int(str(homes.columns[i]).split('-')[1].lstrip('0'))//3) for i in range(len(homes.columns))]
    return homes

In [20]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are different)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p is
    the exact p value returned from scipy.stats.ttest_ind(). The
    value for better is either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    utowns=get_list_of_university_towns()
    utowns.set_index(['State','RegionName'],inplace=True)
    start=get_recession_start()
    bstart=start.split('q')[0]+'q'+str(int(start.split('q')[1])-1)
    bottom=get_recession_bottom()
    homes_sb=convert_housing_data_to_quarters()[[bstart,bottom]]
    u_towns=pd.merge(homes_sb,utowns,how='inner',left_index=True,right_index=True)
    nu_towns=homes_sb.drop(list(u_towns.index))
    u_towns['PQR']=u_towns['2008q2']/u_towns['2009q2']
    nu_towns['PQR']=nu_towns['2008q2']/nu_towns['2009q2']
    p_value=ttest_ind(u_towns['PQR'],nu_towns['PQR'],nan_policy="omit").pvalue
    different = True if(p_value < 0.01) else False
    if(u_towns['PQR'].mean()<nu_towns['PQR'].mean()):
        better='university town'
    else:
        better='non-university town'
    return (different,p_value,better)

In [21]:
run_ttest()

(True, 0.002724063704753125, 'university town')